In [1]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [5]:
# Data Collection From the JGSEE Best Station Database
start_date = '2022-02-01'
end_date = '2022-04-01'

response = requests.get(f'http://localhost:3000/api/floor7?sort=timestamp&keys=date>={start_date},date<={end_date}')
df_database = pd.DataFrame(response.json())
df_database['timestamp'] = df_database['timestamp'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') + timedelta(hours = 7)).strftime("%Y-%m-%dT%H:%M:%S+07:00"))
df_database['date'] = df_database['date'].apply(lambda x: x.split('T')[0])
df_database.rename(columns={'temperature': 'Temperature'}, inplace=True)
for date in df_database['date'].unique():
    print(date)
    df_database.query(f'date == "{date}"').to_csv(f'rawdata/{date}.csv', header=True,index=False)

2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-05
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01


In [3]:
# Data Collection From CSV File
header = ["timestamp", "date", "time", "Blank", "Eeg", "Eed", "Eeb", "EeN", "EeE", "EeS", "EeW", 
            "Evg", "Evd", "Evb", "EvN", "EvE", "EvS", "EvW", "lz", "Thermopile", "Thermistor",
            "Windspeed", "Winddirection", "RH", "Temperature"]

files = [f for f in os.listdir('stationdata/') if not f.startswith('.')]
files = sorted(files)

for file in files:
    file_name = file.split('.')[0]
    file_name = file_name.split('-')
    date = datetime(2000+int(file_name[2]), int(file_name[1]), int(file_name[0]))
    date_name = date.strftime('%Y-%m-%d')
    df_station = pd.read_csv(f'stationdata/{file}',names=header)
    df_station['time'] = df_station['time'].apply(lambda x: f'0{x}' if len(x.split(':')[0])==1 else x)
    df_station['date'] = date_name
    df_station['timestamp'] = df_station['date']+'T'+df_station['time']+'+07:00'
    df_station.to_csv(f'rawdata/{date_name}.csv', header=True,index=False)
    print(date_name)

2022-01-01
2022-02-01
2022-01-02
